In [1]:
import datetime as df
import requests
import random
import math
import mysql.connector
from datetime import datetime, timedelta
import json
from config import api_key, db_credentials

In [4]:
# Connecting to our database ThermoStatData
def connect_to_database():
    host = db_credentials['host']
    user = db_credentials['user']
    password = db_credentials['password']
    database = db_credentials['database']
    cnx = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
    )

    return cnx

cnx = connect_to_database()
cursor = cnx.cursor()

In [5]:
# Creating Synthetic Data 

monthly_avg_temps = [(0, 8), (0, 9), (2, 12), (6, 16), (11, 21), 
                     (14, 25), (16, 27), (16, 27), (13, 23), (10, 19), 
                     (5, 13), (1, 9)]


def select_internal_temp(temp):
    if -30 <= temp <= -20:
        return round(random.uniform(27, 30), 2)
    elif -20 < temp <= -10:
        return round(random.uniform(25, 30), 2)
    elif -10 < temp <= 0:
        return round(random.uniform(20, 27), 2)
    elif 0 < temp <= 10:
        return round(random.uniform(20, 22), 2)
    elif 10 < temp <= 20:
        return round(random.uniform(19, 21), 2)
    elif 20 < temp <= 27:
        return 22
    elif 27 < temp <= 32:
        return round(random.uniform(17, 19), 2)
    elif 32 < temp <= 39:
        return round(random.uniform(16, 18), 2)
    elif temp > 40:
        return 16
    else:
        print(f"Error: unexpected temperature {temp}")
        return -999



def generate_data(start_date, end_date):
    
    # Connect to the database
    cnx = connect_to_database()
    cursor = cnx.cursor()
    
    current_date = start_date
    data = []
    while current_date <= end_date:
        month = current_date.month - 1
        avg_temp = (monthly_avg_temps[month][0] + monthly_avg_temps[month][1]) / 2
        # use a normal distribution to generate a temperature around the average
        temp = round(random.gauss(avg_temp, 2), 2)
        # "feels like" temperature can be up to 2 degrees different
        feels_like = round(temp + random.uniform(-2, 2), 2)
        humidity = random.randint(30, 100)

        if temp > 5:
            # It can't snow if it's above 5 degrees.
            weather = random.choice(['Sunny', 'Rainy', 'Cloudy', 'Windy', 'Stormy','Broken clouds'])
        else:
            weather = random.choice(['Sunny', 'Rainy', 'Cloudy', 'Snowy','Stormy','Broken Clouds'])

        wind_speed = round(random.uniform(0, 30), 2)
        
        internal_temp = select_internal_temp(feels_like)
        
        data.append((current_date, temp, feels_like, humidity, weather, wind_speed, internal_temp))
        current_date += timedelta(days=1)
    return data


# Push Synthetic Data into Database

def push_data_to_db(data):
   
    # Connect to the database
    cnx = connect_to_database()
    cursor = cnx.cursor()
    
    cursor.execute("USE ThermoStatData")

    
    # Create the table if it does not exist
    cursor.execute("""CREATE TABLE IF NOT EXISTS nine (
      date DATE,
      temperature FLOAT,
      feels_like FLOAT,
      humidity INT,
      weather VARCHAR(255),
      wind_speed FLOAT,
      internal_temp FLOAT
    )
    """)        
    add_weather = ("INSERT INTO nine "
                   "(date, temperature, feels_like, humidity, weather, wind_speed, internal_temp) "
                   "VALUES (%s, %s, %s, %s, %s, %s, %s)")

    cursor.executemany(add_weather, data)

    cnx.commit()
    cursor.close()
    cnx.close()

start_date = datetime(2019, 5, 5)
end_date = datetime(2023, 5, 5)
data = generate_data(start_date, end_date)
push_data_to_db(data)

In [7]:
# Creating CSV file from data created in DB (Not necessary but for our reference if we want to train model with CSV)
import csv

def fetch_data_from_db():
    
    # Connect to the database
    cnx = connect_to_database()
    cursor = cnx.cursor()
    
    query = ("SELECT * FROM nine")
    cursor.execute(query)

    with open('nine.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        # Write the header
        writer.writerow(["Date", "Temperature", "Feels Like", "Humidity", "Weather", "Wind Speed", "Internal Temp"])
        # Write the data
        for (date, temperature, feels_like, humidity, weather, wind_speed, internal_temp) in cursor:
            writer.writerow([date, temperature, feels_like, humidity, weather, wind_speed, internal_temp])

    cursor.close()
    cnx.close()

fetch_data_from_db()